# Práctica examen final ETL: datos de vivienda.

### Autor: 
Blanco García, Gabriel: gabriel.blanco@cunef.edu

#### Colegio Universitario de Estudios Financieros 
#### Madrid, enero de 2021

In [3]:
# Operaciones basicas 
import numpy as np
import pandas as pd

# Expresiones regulares
import re

# Spark para trabajar en rdd
import pyspark
from pyspark import SparkContext

# Algunas operaciones en dataframe
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import *

## 1. Inicializar y cargar el contexto Spark

Al trabajar desde Docker, no es necesario volver a instalar Spark, como sí sucede en Google Colab. La inicialización y la carga del contexto Spark son como se muestra a continuación

In [4]:
sc = SparkContext()
conf = sc.getConf()
conf.getAll()

[('spark.driver.host', '172.17.0.2'),
 ('spark.driver.port', '33813'),
 ('spark.rdd.compress', 'True'),
 ('spark.app.id', 'local-1612172159572'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.name', 'pyspark-shell')]

### Lectura de los datos y creación del diccionario de columnas

Estos son los datos con los que se va a trabajar

In [5]:
datos = sc.textFile('../work/data/BDpracticafinalCSV.csv')

Los metadatos se encuentran en la segunda página del fichero excel. La tabla que se muestra a continuación se ha copiado del enunciado del ejerecicio, ya que quedaba mejor que leyendo el excel.

|NOMBRE VARIABLE|DESCRIPTOR|VALORES|
| --- | --- | --- |
|Order|Variable de identificación|1 a 2930|
|MS Zoning|Zona de ubicación de la vivienda|"A rural, C comercial, FV residencial flotante, I industrial, RH residencial alta densidad, RL residencial baja densidad, RM residencial media densidad"|
|Lot Frontage|Longitud de la fachada en pies||
|Lot Area|Superficie de la vivienda en pies cuadrados||
|Land Contour|Contorno del terreno circundante|"Lvl llano, Bnk Tipo bancal, HLS Ladera, Low Depresión"|
|Land Slope|Tipo de pendiente de la vivienda|" Gtl pendiente suave, Mod pendiente moderada, Sev fuerte pendiente"|
|Overall Qual|Grado de calidad de materiales y acabado de la vivienda|De 1 (Muy pobre) a 10 (Excelente)|
|Year Built|Año de construccion de la vivienda||
|Year Remod/Add|Año de última reforma de la vivienda||
|Mas Vnr Type|Tipo de revestimiento exterior|" BrkCmn Ladrillo normal, BrkFace Ladrillo visto, CBlock Bloque de cemento, None Ninguna, Stone Piedra "|
|Exter Qual|Calidad de revestimiento exterior|"Ex Excelente,Gd Bueno,TA Media,Fa Justo"|
|Bsmt Cond|Estado general del sótano|"Ex Excelente, Gd Bueno, TA Media, Fa Justo, Po Pobre,Ss sin sótano"|
|Total Bsmt SF|Superficie del sótano en pies cuadrados|
|Heating QC|Calidad de la calefacción|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Po Pobre"|
|Central Air|Aire acondicionado centralizado|"N No Y Sí"|
|Full Bath|Número de baños completo en planta||
|Half Bath|Número de aseos en planta||
|Bedroom AbvGr|Número de dormitorios en planta||
|Kitchen AbvGr|Número de cocinas en planta||
|Kitchen Qual|Calidad de cocinas|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Po Pobre"|
|TotRms AbvGrd|Número total de habitaciones excluidos los cuartos de baño||
|Garage Cars|Número de plazas de garaje||
|Garage Area|Superficie del garaje||
|Garage Cond|Estado del garaje|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Po Pobre,Sg sin garaje"|
|Pool Area|Superficie de la piscina en pies cuadrados|
|Pool QC|Calidad de la piscina|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Sp no hay piscina"|
|Mo Sold|mes de venta||
|Yr Sold|año de venta||
|SalePrice|precio de venta en dólares||

Lo primero es hacer un `take` de los datos en formato rdd para ver su aspecto

In [6]:
datos.take(5) # split punto y coma

['Order;MS Zoning;Lot Frontage;Lot Area;Land Contour;Land Slope;Overall Qual;Year Built;Year Remod/Add;Mas Vnr Type;Exter Qual;Bsmt Cond;Total Bsmt SF;Heating QC;Central Air;Full Bath;Half Bath;Bedroom AbvGr;Kitchen AbvGr;Kitchen Qual;TotRms AbvGrd;Garage Cars;Garage Area;Garage Cond;Pool Area;Pool QC;Mo Sold;Yr Sold;SalePrice',
 '1;RL;141;31770;Lvl;Gtl;6;1960;1960;Stone;TA;Gd;1080;Fa;Y;1;0;3;1;TA;7;2;528;TA;0;Sp;5;2010;215000',
 '2;RH;80;11622;Lvl;Gtl;5;1961;1961;None;TA;TA;882;TA;Y;1;0;2;1;TA;5;1;730;TA;0;Sp;6;2010;105000',
 '3;RL;81;14267;Lvl;Gtl;6;1958;1958;BrkFace;TA;TA;1329;TA;Y;1;1;3;1;Gd;6;1;312;TA;0;Sp;6;2010;172000',
 '4;RL;93;11160;Lvl;Gtl;7;1968;1968;None;Gd;TA;2110;Ex;Y;2;1;3;1;Ex;8;2;522;TA;0;Sp;4;2010;244000']

Los datos están separados por punto y coma, algo a tener en cuenta a la hora de hacer el split. Ahora se crea un diccionario con las columnas y su posición, para agilizar el trabajo. Se aprovecha la extracción del header para sustiuir los espacios por
guión bajo y pasarlos a minúsculas

Primero se accede al header filtrando solo las que contengan `Order`, que es una de las columnas. Después, con `.map` y `re.sub` se cambian los espacios por guiones bajos. Finalmente, se utiliza collect `collect` (acción) para pasaralo a Python, que se usará para crear el diccionario.

In [7]:
header = datos.\
    filter(lambda x: 'Order' in x).\
    map(lambda x: x.lower()).\
    map(lambda x: re.sub(' ', '_', x)).collect() # Collect para quedármelo en Python

header

['order;ms_zoning;lot_frontage;lot_area;land_contour;land_slope;overall_qual;year_built;year_remod/add;mas_vnr_type;exter_qual;bsmt_cond;total_bsmt_sf;heating_qc;central_air;full_bath;half_bath;bedroom_abvgr;kitchen_abvgr;kitchen_qual;totrms_abvgrd;garage_cars;garage_area;garage_cond;pool_area;pool_qc;mo_sold;yr_sold;saleprice']

In [8]:
# Creacion de lista de strings separadas por punto y coma
header = ''.join(header).split(';')

header # este es el aspecto de lo que serán las keys

['order',
 'ms_zoning',
 'lot_frontage',
 'lot_area',
 'land_contour',
 'land_slope',
 'overall_qual',
 'year_built',
 'year_remod/add',
 'mas_vnr_type',
 'exter_qual',
 'bsmt_cond',
 'total_bsmt_sf',
 'heating_qc',
 'central_air',
 'full_bath',
 'half_bath',
 'bedroom_abvgr',
 'kitchen_abvgr',
 'kitchen_qual',
 'totrms_abvgrd',
 'garage_cars',
 'garage_area',
 'garage_cond',
 'pool_area',
 'pool_qc',
 'mo_sold',
 'yr_sold',
 'saleprice']

Se crean las keys con el header, los values con números de 0 a la longitud del header, y el diccionario que combina ambos elementos.

In [9]:
# Generación del diccionario 
keys_header = header
values_header = range(len(header)) # de 0 a la longitud del header

# Para cada value, lo empareja con su key
for i in values_header:
    diccionario_columnas = dict(zip(keys_header, values_header))

De este modo, se puede acceder fácilmente al número de la columna con la siguiente sintaxis

In [10]:
diccionario_columnas['land_contour'] # ejmplo para land_contour

4

Ahora se dejan los datos preparados para trabajar con ellos, esto es, sin header y separados por punto y coma. Utilizo `filter` para quedarme con aquellas filas que __no__ son el header, y `map(lanmbda x: x.split(';')` para dividir los datos por punto y coma, que es el separador que utilizan

In [11]:
datos = datos.filter(lambda x: 'Order' not in x).\
              map(lambda x: x.split(';'))

Solo se va a trabajar con viviendas. Hay registros de inmuebles comerciales e industriales que no interesan, se eliminan. Son aquellos cuya ms zoning es `I` o `C` (industrial o commercial)

In [12]:
# Localizo la columna con las zonas
zona = diccionario_columnas['ms_zoning']

# Elimino las observaciones de industrial y commercial
datos = datos.filter(lambda x: 'I' not in x[zona] and 'C' not in x[zona]) # utilizo and, porque no quiero ninguno de los dos

### 2.Viviendas distintas en el dataset
Utilizo `distinct` y `count` para contar el numero de viviendas distintas. La diferencia entre el número de viviendas distintas y el número total de viviendas es igual a las viviendas duplicadas. En principio, no tiene sentido que haya viviendas duplicadas. Si no hubiese duplicadas, el total de viviendas únicas coincidiría con el total de observaciones

El indicador de la vivienda, según los metadatos, se encuentra en la columna `order`

In [13]:
order = diccionario_columnas['order'] # está en la posición 0

datos.map(lambda x: x[order]).distinct().count()

2903

Hay 2903 viviendas únicas. Comparo ese dato con el `count` de los datos en general, sin tener en cuenta si son distintos o no.

El total de datos se obteine con count, y son 2909

In [14]:
datos.count()

2909

Efectivamente, hay 6 viviendas duplicadas. Lo que se puede hacer con ellas es eliminarlas. Podemos contar el número de veces que aparece cada vivienda, y eliminar aquellas que aparezcan más de una vez. Lo haco con `reduceByKey`. El proceso es:

1. map para construir un key value de la columna que quiero contar, con el order como key, y 1 como value, para que la cuenta empieze en 1
2. reduceByKey para ir contando los elementos
3. filter para filtrar aquellas viviendas cuyo order aparece más de una vez

In [15]:
datos.\
        map(lambda x: (x[order], 1)).\
        reduceByKey(lambda x, y: x + y).\
        filter(lambda x: x[1] > 1).collect()

[('2930', 2), ('2900', 2), ('2901', 2), ('2898', 2), ('2899', 2), ('2929', 2)]

El resultado coincide con las cuentas hechas con `distinct` y `count`. Una opción para eliminar los duplicados sería filtrar el order de tal modo que no contenga a los order duplicados. El problema es que se perderían tanto el dato original como el duplicado. Como no he encotnrado el modo de hacerlo, lo paso a dataframe, tiro los duplicados, y lo vuelvo a pasar a rdd para seguir trabajando.

In [16]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [17]:
# Genero el dataframe
pandas_dataframe = spark.createDataFrame(datos, 
                      schema=header).toPandas()

In [18]:
# Elimino los duplicados
pandas_dataframe.drop_duplicates(inplace=True) # eliminados en el sitio

In [19]:
# Paso de pandas a dataframe de spark y luego a rdd
sqlContext = SQLContext(sc)

spark_dataframe = sqlContext.createDataFrame(pandas_dataframe)
datos = spark_dataframe.rdd # vuelta a rdd

### 3. Total de inmuebles y precio medio de cada zona

Interpreto que el ejercicio pide obtener, para cada zona, el precio medio de los inmuebles, y el número total de inmuebles de cada zona.

Las columnas de interés son la zona y el precio

In [20]:
# Localizo las columnas
# La zona ya la he creado arriba, no hace falta volver a extrarla
precio = diccionario_columnas['saleprice']

In [21]:
datos.map(lambda x: (x[zona], int(x[precio]))).take(10)

[('RL', 215000),
 ('RH', 105000),
 ('RL', 172000),
 ('RL', 244000),
 ('RL', 189900),
 ('RL', 195500),
 ('RL', 213500),
 ('RL', 191500),
 ('RL', 236500),
 ('RL', 189000)]

Interesa algo así, pero agrupado por zona, y con el precio medio por zona y el conteo

In [22]:
# Este es el total de inmuebles de cada zona.
datos.map(lambda x: (x[zona], int(x[precio]))).countByKey()

defaultdict(int,
            {'A': 2,
             'FV': 139,
             'RH': 27,
             'RL': 2269,
             'RM': 462,
             'Rl': 3,
             'rL': 1})

Hay un problema, y es que rL está mal escrito, aparece varias veces con distitnas escrituras. Para solucionarlo, paso la columna a minúsculas. Aplico combine by key y obtengo, para cada key, que es la zona, la suma de los precios, y el total de casas para cada zona.

In [23]:
precios_zona = datos.map(lambda x: (x[zona].lower(), int(x[precio]))).combineByKey(
    (lambda x: (x, 1)), # valor inicial de la cuenta, siempre 1
    
    (lambda acc, value: (acc[0] + value, acc[1] + 1)), 
    (lambda acc1, acc2: (acc1[0]+acc2[0], acc1[1]+acc2[1])) # se combina el acumulador

)

precios_zona.collectAsMap()

{'a': (94600, 2),
 'fv': (30439186, 139),
 'rh': (3683334, 27),
 'rl': (434786831, 2273),
 'rm': (58573004, 462)}

Ahora para calcular la media, lo unico que hay que hacer es dividir el sumatorio entre el conteo. Aplico map y devuelvo tres elementos: el identificador de la zona, la media, y el total de casas de cada categoría

In [24]:
precios_zona.map(lambda x: (x[0], round(x[1][0] / x[1][1], 2), x[1][1])).collect()

[('fv', 218986.95, 139),
 ('a', 47300.0, 2),
 ('rl', 191283.25, 2273),
 ('rh', 136419.78, 27),
 ('rm', 126781.39, 462)]

La zona fv es la más cara en términos medios

### 4. Media de Total Bsmt SF por cada decada de construcción (year built)

El problema se divide en dos partes:   
1. Extraer la década de cada año
2. Caclcular la media para cada década

Primero localizo las columnas de interés

In [25]:
# Localizo las columnas 
bsmt_sf = diccionario_columnas['total_bsmt_sf']
year_built = diccionario_columnas['year_built']

In [26]:
datos.map(lambda x: (x[bsmt_sf], x[year_built])).take(10) # estos son los datos. Como saco la década?

[('1080', '1960'),
 ('882', '1961'),
 ('1329', '1958'),
 ('2110', '1968'),
 ('928', '1997'),
 ('926', '1998'),
 ('1338', '2001'),
 ('1280', '1992'),
 ('1595', '1995'),
 ('994', '1999')]

Primera cuestión para sacar la década ¿cuántos años hay?

In [27]:
years = datos.map(lambda x: x[year_built]).distinct().collect()
years

['1890',
 '1948',
 '1983',
 '1940',
 '1970',
 '1938',
 '1918',
 '1936',
 '1969',
 '1978',
 '1993',
 '1928',
 '1972',
 '1882',
 '1898',
 '1895',
 '1997',
 '2008',
 '1996',
 '1942',
 '1905',
 '1955',
 '1945',
 '1929',
 '1932',
 '1885',
 '1984',
 '1974',
 '1924',
 '1914',
 '1952',
 '1995',
 '1982',
 '1954',
 '1990',
 '2007',
 '1880',
 '2003',
 '1964',
 '1941',
 '1947',
 '1971',
 '1960',
 '1988',
 '1926',
 '1953',
 '1915',
 '1966',
 '1900',
 '1958',
 '1961',
 '2001',
 '1930',
 '1910',
 '1963',
 '1904',
 '1967',
 '2005',
 '1973',
 '1998',
 '1901',
 '2004',
 '1920',
 '1991',
 '1921',
 '1951',
 '2002',
 '1911',
 '1872',
 '1937',
 '1925',
 '1939',
 '1922',
 '1935',
 '1968',
 '1999',
 '1917',
 '1893',
 '1950',
 '1912',
 '1985',
 '1908',
 '1902',
 '1986',
 '1949',
 '1956',
 '1962',
 '1934',
 '1913',
 '2010',
 '1879',
 '1977',
 '1994',
 '1979',
 '1957',
 '1981',
 '1927',
 '1980',
 '1965',
 '1976',
 '1906',
 '1919',
 '2000',
 '1987',
 '1989',
 '2009',
 '1992',
 '2006',
 '1916',
 '1923',
 '1959',
 

Hay datos desde 1870 hasta pasdo el 2010. Necesito algo que me indique, por un lado, la década, y por otro lado, el siglo, puesto que de usar solo algo que indique la década, la primera década del siglo XX no sería distinguible de la primera década del siglo XXI. 

Los años de la misma década y siglocomparten el segundo y tercer número de su año. Aprovechando que los años están en formato string, se puede acceder al segundo y tercer elemento, para sacar el número que indique el siglo y la década. Por ejemplo, para 1890, la extracción es la siguiente

In [28]:
years[0][1:3] # siglo XIX, decada 9

'89'

De este modo, se identifican las decadas y el siglo, puesto que el primer dígito hace referencia al siglo (en centenares, claro) y el segundo dígito a la década. 1__99__5 y 1__99__2 quedan correctamente capturados en el mismo grupo. Y se diferencian de 1__88__5 y 1__88__2. 

Este sistema solo da probelmas si se manejan datos con diferencias temporales de 1.000 o más años, puesto que con 2 números se pueden identificar 99 décadas diferentes. Como no es el caso de este dataset, el sistema es conveniente.

Aplicando esta idea a toda la columna, el resultado sería el siguiente

In [29]:
por_decada = datos.map(lambda x: (x[year_built][1:3], int(x[bsmt_sf])))

por_decada.take(10)

[('96', 1080),
 ('96', 882),
 ('95', 1329),
 ('96', 2110),
 ('99', 928),
 ('99', 926),
 ('00', 1338),
 ('99', 1280),
 ('99', 1595),
 ('99', 994)]

Ahora es el mimso procedimiento de antes, `combineByKey`

In [30]:
bsmt_decada = por_decada.map(lambda x: (x[0], x[1])).combineByKey(
    (lambda x: (x, 1)), # valor inicial de la cuenta, siempre 1
    
    (lambda acc, value: (acc[0] + value, acc[1] + 1)), # el contador
    (lambda acc1, acc2: (acc1[0]+acc2[0], acc1[1]+acc2[1]))  # el acumulador, va sumando los valores de bsmt_sf

)

bsmt_decada.collectAsMap()

{'00': (1021917, 780),
 '01': (4720, 3),
 '87': (2283, 3),
 '88': (6688, 8),
 '89': (10627, 12),
 '90': (25925, 36),
 '91': (81195, 103),
 '92': (158448, 190),
 '93': (82381, 107),
 '94': (105332, 149),
 '95': (324379, 337),
 '96': (386514, 357),
 '97': (346791, 364),
 '98': (130320, 120),
 '99': (375481, 334)}

Y ahora la media dividiendo el sumatorio por el conteo. Podemos usar sort by key para ordenar los resultados. Lo único que hay que tener en cuenta es que las dos primeras décadas del siglo XXI aparecen primero, porque su indicador es menor que el resto, aunque cronológicamente sucedan después

In [31]:
bsmt_decada.map(lambda x: (x[0], round(x[1][0] / x[1][1], 2))).sortByKey().collect()

[('00', 1310.15),
 ('01', 1573.33),
 ('87', 761.0),
 ('88', 836.0),
 ('89', 885.58),
 ('90', 720.14),
 ('91', 788.3),
 ('92', 833.94),
 ('93', 769.92),
 ('94', 706.93),
 ('95', 962.55),
 ('96', 1082.67),
 ('97', 952.72),
 ('98', 1086.0),
 ('99', 1124.19)]

Esa es la media de la superficie del sótano por cada década.

### 5. ¿Cuál es la decada de construcción con viviendas mejor acondicionadas para el frío (Heating QC)?

Se va a aplicar el mismo procedimiento para obtener la década, explicado en el ejercicio anterior.

In [32]:
# Localizo la columna de interés
revestimiento = diccionario_columnas['heating_qc']

Primero obtengo el revesitmiento y la década tal que así

In [33]:
revestimiento_decada = datos.map(lambda x: (x[year_built][1:3], x[revestimiento]))

revestimiento_decada.take(7)

[('96', 'Fa'),
 ('96', 'TA'),
 ('95', 'TA'),
 ('96', 'Ex'),
 ('99', 'Gd'),
 ('99', 'Ex'),
 ('00', 'Ex')]

No sirve de nada así. Aplico encoding a los revesitmientos con una puntuación del 1 al 5 para poder trabajar con números. Voy a generar un diccionario cuya key sean los tipos de revestimiento, y values de 1 a 5. De este modo, evito usar if-else

In [34]:
keys_revestimientos = datos.map(lambda x: x[revestimiento]).distinct().collect()

keys_revestimientos

['Fa', 'TA', 'Gd', 'Ex', 'Po']

Quiero que estén ordenados de peor a mejor, lo reordeno aplicando list comprehension

In [35]:
orden = [4, 0, 1, 2, 3] # el orden en que los quiero respecto a su posición actual

keys_revestimientos = [keys_revestimientos[i] for i in orden] # la ordenación

keys_revestimientos # resultado

['Po', 'Fa', 'TA', 'Gd', 'Ex']

Ahora genero los values, puntuaciones de 1 a 5

In [36]:
values_revestimiento = range(1, 6) # los valores del diccionario

Y creo el diccionario

In [37]:
# Genero el diccionario
for i in values_revestimiento:
    diccionario_revestimiento = dict(zip(keys_revestimientos, values_revestimiento))

diccionario_revestimiento # resultado

{'Ex': 5, 'Fa': 2, 'Gd': 4, 'Po': 1, 'TA': 3}

¿Por qué empleo el diccionario? Porque lo voy a aplicar en una función de tal manera que me evite el if else 5 veces. Al usar el diccionario para cambiar por puntuación, el resultado es más directo.  

Además, no estoy seguro, pero creo que es más eficiente computacionalmente, puesto que el if else con 5 condiciones implicaría hacer varias comprobaciones hasta encontrar el match

In [38]:
def puntuacion_revestimiento(revestimiento):
    return diccionario_revestimiento[revestimiento] # la función es así de simple

Nota: realmente no haría falta definir la función, bastaría con usar la propia columna del rdd como key en el diccionario. Lo hago porque el código queda más limpio.

Este sería el resultado de aplicar la función, compruebo que funciona y devuelve los resultados esperados

In [39]:
datos.map(lambda x: (x[revestimiento], puntuacion_revestimiento(x[revestimiento]))).take(10)

[('Fa', 2),
 ('TA', 3),
 ('TA', 3),
 ('Ex', 5),
 ('Gd', 4),
 ('Ex', 5),
 ('Ex', 5),
 ('Ex', 5),
 ('Ex', 5),
 ('Gd', 4)]

Ahora genero una estrucutra con las décadas, como antes, y con las __puntuaciones__ del revesitmineto, en lugar de la string

In [40]:
puntuacion_decadas = datos.map(lambda x: (x[year_built][1:3], # obtención de la década igual que antes 
                                          puntuacion_revestimiento(x[revestimiento]))) # encoding del revestimiento

puntuacion_decadas.take(10)

[('96', 2),
 ('96', 3),
 ('95', 3),
 ('96', 5),
 ('99', 4),
 ('99', 5),
 ('00', 5),
 ('99', 5),
 ('99', 5),
 ('99', 4)]

Utilizando `combineByKey` sobre la década y la puntuación, genero una estructura con la década, la suma de la puntuación, y la cuenta de pisos de esa década

In [41]:
datos_decadas = datos.map(lambda x: (x[year_built][1:3], puntuacion_revestimiento(x[revestimiento]))).combineByKey(
    (lambda x: (x, 1)), # valor inicial de la cuenta, siempre 1
    
    (lambda acc, value: (acc[0] + value, acc[1] + 1)), # el contador
    (lambda acc1, acc2: (acc1[0]+acc2[0], acc1[1]+acc2[1])) 

)

datos_decadas.collect()

[('88', (31, 8)),
 ('00', (3872, 780)),
 ('99', (1577, 334)),
 ('01', (15, 3)),
 ('97', (1257, 364)),
 ('95', (1261, 337)),
 ('94', (571, 149)),
 ('98', (450, 120)),
 ('91', (404, 103)),
 ('93', (419, 107)),
 ('96', (1316, 357)),
 ('87', (9, 3)),
 ('92', (693, 190)),
 ('89', (48, 12)),
 ('90', (142, 36))]

Y ahora saco la puntuación media de cada década, como siempre, dividiento la suma entre la cuenta. Ordeno los resultados

In [42]:
datos_decadas.map(lambda x: (round(x[1][0] / x[1][1], 2), x[0])).sortByKey(ascending=False).collect()

[(5.0, '01'),
 (4.96, '00'),
 (4.72, '99'),
 (4.0, '89'),
 (3.94, '90'),
 (3.92, '91'),
 (3.92, '93'),
 (3.88, '88'),
 (3.83, '94'),
 (3.75, '98'),
 (3.74, '95'),
 (3.69, '96'),
 (3.65, '92'),
 (3.45, '97'),
 (3.0, '87')]

La década en la que los pisos están mejor revestidos contra el frío, en términos medios, es la segunda del 2000, años del 2010 en adelante.

### 6. Cuáles son las 10 viviendas que se vendieron por un precio más elevado por metro cuadrado en el año 2009

Entiendo que el ejercicio hace referencia solo al año 2009, así que empiezo filtrando los datos. Entiendo que sólo se considera la superficie de la vivienda, porque dice vivienda, no la de la piscina o garaje, pero el resultado sería el mismo sumando las superficies.

In [43]:
year_sold = diccionario_columnas['yr_sold'] # año de venta

datos_2009 = datos.filter(lambda x: '2009' in x[year_sold]) # filtro y me quedo con 2009

Ahora hay que calcular el precio en términos del metro cuadrado, dividiendo el precio entre los metros cuadrados de la vivienda. __Cuidado__, los datos están en pies cuadrados. Hay que pasar a metro cuadrado. Un pie cuadrado equivale a 0,092903 metros cuadrados

In [44]:
# Columnas de interés
precio = diccionario_columnas['saleprice']
superficie = diccionario_columnas['lot_area']

Hago la operación. Incluyo el indicador de la vivienda para ver cuales son exactamente. El precio entre __metros__ cuadrados se utiliza como key para poder usar `sortByKey`, y mostrar los 10 más altos.

In [45]:
datos_2009.map(lambda x: (round(int(x[precio]) / (float(x[superficie])*0.092903), 2), 
                          x[order])).sortByKey(False).take(10)

[(1049.29, '566'),
 (1026.71, '936'),
 (991.03, '934'),
 (821.06, '464'),
 (807.96, '935'),
 (765.65, '408'),
 (756.04, '407'),
 (717.59, '405'),
 (714.28, '933'),
 (711.19, '403')]

La vivienda más cara por metro cuadrado vendida en 2009 es la 566

### 7. Media anual por zonas del precio de venta y metros cuadrados.

De nuevo, entiendo que hay que considerar solo la superficie de la vivienda. Primero calculo el precio en términos de superficie para cada casa

In [46]:
# Genero los datos, tomo 2 decimales en precio/superficie, convierto a pies
precios_year_zona = datos.map(lambda x: (x[year_built], 
                                         x[zona].lower(), 
                                         round(int(x[precio]) / (float(x[superficie])*0.092903), 2)))

precios_year_zona.take(8)

[('1960', 'rl', 72.84),
 ('1961', 'rh', 97.25),
 ('1958', 'rl', 129.77),
 ('1968', 'rl', 235.34),
 ('1997', 'rl', 147.8),
 ('1998', 'rl', 210.9),
 ('2001', 'rl', 467.09),
 ('1992', 'rl', 411.85)]

Ahora utilizo `combineByKey` para calcular la suma por año y zona y el conteo. La diferencia es que ahora la key es una tupla de años y zona, en lugar de un solo valor

In [47]:
precios_zona_year_combinado = precios_year_zona.map(lambda x: ((x[0], x[1]), x[2])).combineByKey(
    (lambda x: (x, 1)), # valor inicial de la cuenta, siempre 1
    
    (lambda acc, value: (acc[0] + value, acc[1] + 1)), # el contador
    (lambda acc1, acc2: (acc1[0]+acc2[0], acc1[1]+acc2[1]))
)

precios_zona_year_combinado.take(5)

[(('1916', 'rl'), (666.08, 4)),
 (('1946', 'rl'), (1411.32, 10)),
 (('1915', 'rm'), (2715.87, 13)),
 (('1930', 'rm'), (3469.48, 18)),
 (('1904', 'rm'), (162.39, 1))]

Y ahora saco la media como siempre. La pongo como key para poder ordenar, aunque el ejercicio no pide ordenarlos. El resultado es el siguiente

In [48]:
precios_zona_year_combinado.map(lambda x: (round(x[1][0] / x[1][1], 2), x[0])).sortByKey(False).take(10)

[(749.82, ('1980', 'rm')),
 (734.49, ('2007', 'rm')),
 (684.29, ('1999', 'fv')),
 (669.86, ('1998', 'fv')),
 (648.72, ('2000', 'fv')),
 (628.74, ('2006', 'rm')),
 (626.47, ('1972', 'rm')),
 (620.76, ('1971', 'rm')),
 (568.73, ('1973', 'rm')),
 (568.54, ('1970', 'rm'))]

La zona más cara en 1980 es rm, también lo fue en 2007. Se trata de residencias en zonas de densidad medias

### 8. Podrías decirme el total de recaudación de las casas de revistimiento (Mas Vnr Type) de piedra con respecto a las de ladrillo? ¿Hay diferencia significativa?

Considero ladrillo visto y ladrillo normal en la misma categoría. Para piedra considero solo Stone. También interpreto que el total recaudado es la suma de los precios, en valor absoluto

In [49]:
# columna de interés 
tipo =  diccionario_columnas['mas_vnr_type']

Primero compruebo el estado de las categorías

In [50]:
datos.map(lambda x: x[tipo]).distinct().collect() 

['', 'None', 'Stone', 'BrkCmn', 'BrkFace', 'CBlock']

Genero dos subset con cada uno de los tipos, uno con ladrillos y otro con piedra.

In [51]:
casas_ladrillo = datos.filter(lambda x: 'BrkCmn' in x[tipo] or 'BrkFace' in x[tipo])
casas_piedra = datos.filter(lambda x: 'Stone' in x[tipo])

Calculo la suma de los precios para cada rdd e imprimo la información

In [52]:
recaudacion_ladrillo = casas_ladrillo.map(lambda x: int(x[precio])).sum()
recaudacion_piedra = casas_piedra.map(lambda x: int(x[precio])).sum()

# Imprimo los datos
print('El total de recaudación de las casas de ladrillo es de', recaudacion_ladrillo, 'dólares.') 
print('El total de recaudación de las casas de piedra es de', recaudacion_piedra, 'dólares') 

print()

print('La diferencia entre ambos tipos de casa es de', recaudacion_ladrillo - recaudacion_piedra, 
      'dólares, sinedo la recaudación de las casas de ladrillo muy superior a la recaudación de las casas de piedra')

El total de recaudación de las casas de ladrillo es de 189007736 dólares.
El total de recaudación de las casas de piedra es de 64876277 dólares

La diferencia entre ambos tipos de casa es de 124131459 dólares, sinedo la recaudación de las casas de ladrillo muy superior a la recaudación de las casas de piedra


### 9. Cuánto son más caras las viviendas con 2 cocinas, con 2 o más plazas de garaje que las que tienen 1 cocina y 1 plaza de garaje? Comparar medias y cuartiles de ambos caso

Preparo los datos, primero filtro las que tienen 2 cocinas y 2 o más plazas de garaje

In [53]:
# Columnas de interés 
cocinas = diccionario_columnas['kitchen_abvgr']
plazas_garaje = diccionario_columnas['garage_cars']

In [54]:
# Genero los subsets filtrando acorde con el enunciado.
casas_big = datos.filter(lambda x: (int(x[cocinas]) == 2) and (int(x[plazas_garaje]) >= 2))

# Filtro los garajes como string, porque para las casas de una sola cocina, algunas tenían nulos 
# en el número de garajes. De este modo, evito los nulos
casas_small = datos.filter(lambda x: (int(x[cocinas]) == 1) and ('1' in x[plazas_garaje])) 

Defino una función para calcular la media. La función, al recorrer la lista, hace dos cosas:
1. Sumar los elementos
2. Contar los elementos  

De este modo, se consiguen los dos elementos necesarios para la media (sumatorio y conteo) en un solo recorrido del rdd. La función genera una tupla cuyo primer elemento es el total, y cuyo segundo elemento es el sumatorio. Lo que devuelve es la media, que es el resultado de dividir el pimer elemento de la tupla entre el segundo. Se usan 2 decimales.

In [67]:
# Utilizo una función para no repetir exacatemente el mismo código dos veces
def media(indicador_columna, datos):
    
    columna = datos.map(lambda x: float(x[indicador_columna])) # extrae la columna de interes como float
    
    suma_columna = columna.aggregate( # y hace el agregado
    (0,0), # lista vacía como valor inicial
        
    (lambda acc, value: (acc[0] + value, acc[1] + 1)), # a cada valor, le suma el siguiente. acc[1] + 1 para que empieze en 1
                                                       # la cuenta
    (lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1]))

    )
    
    media = round(suma_columna[0] / suma_columna[1], 2)
    
    return media

Aplico la función e imprimo los datos

In [68]:
# Calculo la media para cada subset
media_big = media(precio, casas_big)
media_small = media(precio, casas_small) 

# Imprimo los datos
print('El precio medio de las casas con 2 cocinas y 2 o más plazas de garaje es de', media_big, '$')
print('El precio medio de las casas con 1 cocina y una plaza de garaje es de', media_small, '$')
print()
print('La diferencia de precios medios es de', round(media_big - media_small, 2), 'dólares. Es significativa')

El precio medio de las casas con 2 cocinas y 2 o más plazas de garaje es de 145124.28 $
El precio medio de las casas con 1 cocina y una plaza de garaje es de 128155.36 $

La diferencia de precios medios es de 16968.92 dólares. Es significativa


Vemos que la diferencia en precios entre los dos tipos de casas es significativa, de más de 16.000 dólares. Es coherente, puesto que las casas con 2 cocinas y dos o más plazas de garaje son más grandes que las de una cocina y una plaza de garaje.

Para calcular los cuartiles, habría que ordenar los precios de menor a mayor, y partirlos en 3 trozos que dejasen cuartas partes de los datos a cada lado. A raíz de lo comentado en la tutoría, dejo solo el cálculo de la media, puesto que se comentó que los cuartiles no eran absolutamente necesarios si no nos salían.